In [1]:
!pip install pandas scikit-learn matplotlib seaborn


Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd

df = pd.read_csv("ml_customer_features.csv")
df.head()


,customer_id,total_orders,total_revenue,avg_order_value,recency_days,tenure_days,churn_label
0,12713,1,848.55,848.55,0,0,0
1,14569,1,227.39,227.39,1,0,0
2,15520,1,343.50,343.50,1,0,0
3,13298,1,360.00,360.00,1,0,0
4,13436,1,196.89,196.89,1,0,0


In [8]:
X = df.drop(columns=["churn_label", "recency_days"])
y = df["churn_label"]



In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    random_state=42,
    stratify=y
)


In [10]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [11]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [12]:
from sklearn.metrics import classification_report, roc_auc_score

y_pred = model.predict(X_test_scaled)
y_prob = model.predict_proba(X_test_scaled)[:, 1]

print(classification_report(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_prob))


              precision    recall  f1-score   support

           0       0.79      0.74      0.76       867
           1       0.54      0.61      0.57       435

    accuracy                           0.69      1302
   macro avg       0.66      0.67      0.67      1302
weighted avg       0.71      0.69      0.70      1302

ROC AUC: 0.7724694746052579


In [13]:
import pandas as pd

feature_importance = pd.DataFrame({
    "feature": X.columns,
    "coefficient": model.coef_[0]
}).sort_values(by="coefficient", ascending=False)

feature_importance


,feature,coefficient
0,customer_id,0.019208
3,avg_order_value,-0.101590
2,total_revenue,-0.767062
4,tenure_days,-0.917531
1,total_orders,-1.064956


In [14]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

features_for_clustering = df.drop(columns=["churn_label", "recency_days"])

scaler = StandardScaler()
X_scaled = scaler.fit_transform(features_for_clustering)


In [15]:
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(X_scaled)

df["customer_segment"] = clusters
df["customer_segment"].value_counts()


customer_segment
0    1529
2    1487
1    1322
Name: count, dtype: int64

In [16]:
df.groupby("customer_segment").mean()


,customer_id,total_orders,total_revenue,avg_order_value,recency_days,tenure_days,churn_label
customer_segment,,,,,,,
0,16819.242642,2.028777,638.491499,317.232090,119.560497,54.015697,0.451929
1,15296.685325,9.388048,5083.731921,557.989150,28.158850,300.638427,0.046899
2,13741.983860,2.030262,816.718764,400.560755,120.591796,58.677202,0.468056
